In [1]:
from tls_crypt import *

In [2]:
def str2list(s) :
    r = []
    for c in s: r.append(ord(c))
    return r

def list2str(l) :
    r = ''
    for c in l: r += chr(c)
    return r

def hex2list(s):
    assert len(s) % 16 == 0
    l = []
    str = ''
    for c in s:
        str += c
        if len(str) == 2:
            l.append(int(str, 16))
            str = ''
    return l

def hexstr2list(s):
    li = s.split(' ')
    return [int(i, 16) for i in li]

def hexprint(li):
    print('[', end='')
    for i in li : print('{:02x}'.format(i), end=',')
    print('\b]')
    
def list2int(li):
    k = 0;
    for i in li:
        k *= 0x100;
        k += i
    return k

def int2list(k):
    r = []
    while k > 0:
        r.insert(0, k%0x100)
        k = k // 0x100
    return r

In [3]:
aes = AES()
hkdf = HKDF()
x25519 = X25519()

In [4]:
a = '16 03 01 02 00 01 00 01 fc 03 03 b6 27 34 3b 77 bf 1f 66 54 f3 d4 68 5a 08 b2 20 75 91 7d 03 e5 1b dc b8 67 06 dc c5 70 be 7e ea 20 d6 1c ac 4b fe ec 6e 41 ea 45 b5 e6 da 53 a2 82 86 f6 0c d7 9f da ad 88 9b ef 09 fd f9 9b 2d b4 00 22 4a 4a 13 01 13 02 13 03 c0 2b c0 2f c0 2c c0 30 cc a9 cc a8 c0 13 c0 14 00 9c 00 9d 00 2f 00 35 00 0a 01 00 01 91 6a 6a 00 00 00 00 00 0e 00 0c 00 00 09 6c 6f 63 61 6c 68 6f 73 74 00 17 00 00 ff 01 00 01 00 00 0a 00 0a 00 08 7a 7a 00 1d 00 17 00 18 00 0b 00 02 01 00 00 23 00 00 00 10 00 0e 00 0c 02 68 32 08 68 74 74 70 2f 31 2e 31 00 05 00 05 01 00 00 00 00 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 12 00 00 00 33 00 2b 00 29 7a 7a 00 01 00 00 1d 00 20'
b = '00 2d 00 02 01 01 00 2b 00 0b 0a 6a 6a 03 04 03 03 03 02 03 01 00 1b 00 03 02 00 02 aa aa 00 01 00 00 15 00 98 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 29 00 33 00 0e 00 08'
c = 'f4 cd f0 05 00 21 20 6e e7 a9 a7 f6 50 90 f9 7e 89 88 a4 71 4b d0 d0 5f d7 a8 cc 96 e4 30 61 81 ac e8 70 70 2d 1a a7'

pri = 0xbff91188283846dd6a2134ef7180ca2b0b14fb10dce707b5098c0dddc813b2df
pub = 0xe4ffb68ac05f8d96c99da26698346c6be16482badddafe051a66b4f18d668f0b
print ('{0:#x}'.format(x25519.mul_g(pri)))

resumption_psk = 0xA3DC69782BC632358DF50BEBE7E5375EEB337A503EAF43E6A68EB33C1D00E7E4
ticket_id = ' 26 c7 4c 9b 77 a9 6d 21 '
client_hello = hexstr2list(a) + int2list(pub) + hexstr2list(b + ticket_id + c)
hkdf.zero_salt()
early = hkdf.extract(int2list(resumption_psk))
hexprint(early)

0xe4ffb68ac05f8d96c99da26698346c6be16482badddafe051a66b4f18d668f0b
[80,62,9c,e1,bf,7c,53,af,a6,57,de,90,81,0a,04,bc,a3,05,a1,9a,cb,31,9e,8d,e7,5e,0d,89,4d,60,92,cf,]


In [16]:
client = TLS_client('localhost', 4433)
client.send(client_hello)

hexprint(client.recv())


[14,03,03,00,01,01,]


In [17]:
hexprint(client.recv())

[17,03,03,00,43,84,2c,44,6d,ba,de,92,1d,c3,4d,d9,fd,af,c1,ca,ff,b2,4f,cb,39,de,21,06,17,a1,00,65,ed,94,46,67,b9,5c,80,71,f7,3c,f5,11,25,3a,a2,2b,8c,9c,ed,22,3f,72,7f,14,6c,5c,39,0d,b8,28,d0,42,4b,b0,58,1d,76,9f,0d,aa,]


In [18]:
hexprint(client.recv())

[16,03,03,00,80,02,00,00,7c,03,03,f3,ec,3e,56,97,ba,c8,57,8d,f2,92,be,3c,62,c1,c3,c2,4d,11,e7,60,61,f4,05,63,c2,79,0c,f6,9d,bd,99,20,d6,1c,ac,4b,fe,ec,6e,41,ea,45,b5,e6,da,53,a2,82,86,f6,0c,d7,9f,da,ad,88,9b,ef,09,fd,f9,9b,2d,b4,13,01,00,00,34,00,2b,00,02,03,04,00,33,00,24,00,1d,00,20,2f,e5,7d,a3,47,cd,62,43,15,28,da,ac,5f,bb,29,07,30,ff,f6,84,af,c4,cf,c2,ed,90,99,5f,58,cb,3b,74,00,29,00,02,00,00,]


In [19]:
hexprint(client.recv())

[14,03,03,00,01,01,]


In [20]:
hexprint(client.recv())

[17,03,03,00,43,6a,14,13,87,f0,5f,52,55,de,46,96,8d,4d,55,a0,b8,9f,c7,b7,91,b0,9a,cf,19,0a,a5,44,ef,d8,1b,39,52,7a,60,eb,b4,50,df,5e,05,38,f8,bf,cd,c1,3c,71,43,18,04,48,0f,a9,fa,86,66,eb,7c,c6,54,0b,f6,83,54,38,0e,0d,]


In [5]:
server_pub = '2f e5 7d a3 47 cd 62 43 15 28 da ac 5f bb 29 07 30 ff f6 84 af c4 cf c2 ed 90 99 5f 58 cb 3b 74'
ecdhe = x25519.mul(pri, list2int(hexstr2list(server_pub)))


In [7]:
print('{:x}'.format(ecdhe))


246cd6b4a16eea301dc8bb99c91fa97bcceead1540d40680f6ece4f3d15dd552


In [30]:
hkdf.salt(early)
tmp = hkdf.derive_secret('derived', [])
hkdf.salt(tmp)
handshake_secret = hkdf.extract(int2list(ecdhe))
hexprint(handshake_secret)

hkdf.salt(handshake_secret)
s_hs_traffic = hkdf.derive_secret('s hs traffic', hexstr2list('01 00 01 fc 03 03 b6 27 34 3b 77 bf 1f 66 54 f3 d4 68 5a 08 b2 20 75 91 7d 03 e5 1b dc b8 67 06 dc c5 70 be 7e ea 20 d6 1c ac 4b fe ec 6e 41 ea 45 b5 e6 da 53 a2 82 86 f6 0c d7 9f da ad 88 9b ef 09 fd f9 9b 2d b4 00 22 4a 4a 13 01 13 02 13 03 c0 2b c0 2f c0 2c c0 30 cc a9 cc a8 c0 13 c0 14 00 9c 00 9d 00 2f 00 35 00 0a 01 00 01 91 6a 6a 00 00 00 00 00 0e 00 0c 00 00 09 6c 6f 63 61 6c 68 6f 73 74 00 17 00 00 ff 01 00 01 00 00 0a 00 0a 00 08 7a 7a 00 1d 00 17 00 18 00 0b 00 02 01 00 00 23 00 00 00 10 00 0e 00 0c 02 68 32 08 68 74 74 70 2f 31 2e 31 00 05 00 05 01 00 00 00 00 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 12 00 00 00 33 00 2b 00 29 7a 7a 00 01 00 00 1d 00 20 e4 ff b6 8a c0 5f 8d 96 c9 9d a2 66 98 34 6c 6b e1 64 82 ba dd da fe 05 1a 66 b4 f1 8d 66 8f 0b 00 2d 00 02 01 01 00 2b 00 0b 0a 6a 6a 03 04 03 03 03 02 03 01 00 1b 00 03 02 00 02 aa aa 00 01 00 00 15 00 98 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 29 00 33 00 0e 00 08 26 c7 4c 9b 77 a9 6d 21 f4 cd f0 05 00 21 20 6e e7 a9 a7 f6 50 90 f9 7e 89 88 a4 71 4b d0 d0 5f d7 a8 cc 96 e4 30 61 81 ac e8 70 70 2d 1a a7 02 00 00 7c 03 03 f3 ec 3e 56 97 ba c8 57 8d f2 92 be 3c 62 c1 c3 c2 4d 11 e7 60 61 f4 05 63 c2 79 0c f6 9d bd 99 20 d6 1c ac 4b fe ec 6e 41 ea 45 b5 e6 da 53 a2 82 86 f6 0c d7 9f da ad 88 9b ef 09 fd f9 9b 2d b4 13 01 00 00 34 00 2b 00 02 03 04 00 33 00 24 00 1d 00 20 2f e5 7d a3 47 cd 62 43 15 28 da ac 5f bb 29 07 30 ff f6 84 af c4 cf c2 ed 90 99 5f 58 cb 3b 74 00 29 00 02 00 00'))
hexprint(s_hs_traffic)

hkdf.salt(s_hs_traffic)
key = hkdf.expand_label('key', '', 16)
iv = hkdf.expand_label('iv', '', 12)
aes.key(list2int(key))
aes.iv(list2int(iv))
aes.aad([23, 3, 3, 0, 0x43])

r = aes.decrypt(hexstr2list('6a 14 13 87 f0 5f 52 55 de 46 96 8d 4d 55 a0 b8 9f c7 b7 91 b0 9a cf 19 0a a5 44 ef d8 1b 39 52 7a 60 eb b4 50 df 5e 05 38 f8 bf cd c1 3c 71 43 18 04 48'))
hexprint(r[0])
hexprint(r[1])

[c3,0f,10,1d,57,71,83,52,b8,bc,20,b0,52,ad,6b,a4,62,23,53,0d,71,88,7b,d9,1a,ed,98,69,76,85,e2,4a,]
[53,a0,91,03,28,d4,8d,bd,67,ac,4a,87,d3,eb,55,81,a1,e1,75,b1,88,8f,12,bf,d3,c8,a2,16,63,61,54,b4,]
[08,00,00,0a,00,08,00,0a,00,04,00,1d,00,17,14,00,00,20,39,b7,86,de,39,13,45,eb,f9,cf,a1,4d,56,43,ad,c1,e1,cf,30,43,3f,e0,57,d1,14,29,f2,5c,35,03,f1,28,16,]
[0f,a9,fa,86,66,eb,7c,c6,54,0b,f6,83,54,38,0e,0d,]


In [32]:
auth = hexstr2list('0f a9 fa 86 66 eb 7c c6 54 0b f6 83 54 38 0e 0d')